---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [5]:
def graph_identification():
    df=pd.DataFrame()
    # Your Code Here
    
    return ['PA','SW_L','SW_L','PA','SW_H'] # Your Answer Here
#graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [6]:
G = nx.read_gpickle('email_prediction.txt')

#print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [66]:
def salary_predictions():
    from sklearn.ensemble import RandomForestClassifier as RFC
    #from sklearn.model_selection import GridSearchCV as GSCV
    #r=nx.Graph()
    
    df=pd.DataFrame(G.nodes(data=True))
    df['D']=[x['Department'] for x in df[1]]
    df['MS']=[x['ManagementSalary'] for x in df[1]]
    df=df[['D','MS']]
    lcc=nx.degree_centrality(G)
    df['lcc']=[lcc[x] for x in df.index]
    #cc=nx.closeness_centrality(G)
    #df['cc']=[cc[x] for x in df.index]
    # Your Code Here
    n=[x for x,y in G.nodes(data=True) if (y['ManagementSalary']==0 or y['ManagementSalary']==1)]
    nt=[x for x,y in G.nodes(data=True) if (y['ManagementSalary']!=0 and y['ManagementSalary']!=1)]
    
    xtrain=df.loc[n,['lcc','D']]
    ytrain=df.loc[n,'MS']
    xtest=df.loc[nt,['lcc','D']]
    
    #r=RFC()
    #pg={'n_estimators':[10,50,100], 'max_depth':[3,5,7]}
    #gs=GSCV(r,param_grid=pg,scoring='roc_auc').fit(xtrain,ytrain)
    rf=RFC(n_estimators=100,max_depth=5).fit(xtrain,ytrain)
    ans=pd.Series(data=rf.predict_proba(xtest)[:,1],index=nt)
    return   ans # Your Answer Here
salary_predictions()

1       0.097173
2       0.801990
5       0.969070
8       0.151776
14      0.154984
18      0.108186
27      0.233776
30      0.225733
31      0.297996
34      0.137133
37      0.224351
40      0.269936
45      0.139410
54      0.463367
55      0.381071
60      0.227344
62      0.978385
65      0.449353
77      0.023747
79      0.127503
97      0.028936
101     0.082841
103     0.216330
108     0.185553
113     0.526351
122     0.027743
141     0.235953
142     0.978385
144     0.043912
145     0.180930
          ...   
913     0.043065
914     0.060457
915     0.045442
918     0.059569
923     0.031314
926     0.017806
931     0.017806
934     0.052234
939     0.042467
944     0.036103
945     0.029270
947     0.029797
950     0.149574
951     0.022409
953     0.049912
959     0.026931
962     0.032686
963     0.133769
968     0.020961
969     0.023479
974     0.027085
984     0.054287
987     0.036899
989     0.027798
991     0.040217
992     0.036972
994     0.040579
996     0.0211

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [34]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [ ]:
def new_connections_predictions():
    from sklearn.ensemble import GradientBoostingClassifier
    
    future_connections['pref_attachment'] = [list(nx.preferential_attachment(G, [node_pair]))[0][2]
                                             for node_pair in future_connections.index]
    future_connections['comm_neighbors'] = [len(list(nx.common_neighbors(G, node_pair[0], node_pair[1]))) 
                                            for node_pair in future_connections.index]
    train_data = future_connections[~future_connections['Future Connection'].isnull()]
    test_data = future_connections[future_connections['Future Connection'].isnull()]
    clf = GradientBoostingClassifier()
    clf.fit(train_data[['pref_attachment','comm_neighbors']].values, train_data['Future Connection'].values)
    preds = clf.predict_proba(test_data[['pref_attachment','comm_neighbors']].values)[:,1]
    return pd.Series(preds, index=test_data.index)
    
    
new_connections_predictions()